# PromptTemplat

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os
print(os.environ['LANGSMITH_PROJECT'])

CH02-Prompt


프롬프트 단계는 검색기에서 검색된 문서들을 바탕으로 **언어 모델이 사용할 질문이나 명령을 생성하는 과정** 입니다. 이 단계는 검색된 정보를 바탕으로 최종 사용자의 질문에 가장 잘 대응할 수 있는 응답을 생성하기 위해 필수적인 단계입니다.

## 프롬프트의 필요성

1. 문맥(Context) 설정 : </br>
프롬프트는 언어 모델이 특정 문맥에서 작동하도록 설정하는 역할을 합니다. 이를 통해 모델은 제공된 정보를 바탕으로 보다 정확하고 관련성 높은 답변을 생성할 수 있습니다.

2. 정보 통합 : </br>
여러 문서에서 검색된 정보는 서로 다른 관점이나 내용을 포함할 수 있습니다. 프롬프트 단계에서 이러한 정보를 통합하고, 모델이 이를 효율적으로 활용할 수 있는 형식으로 조정합니다.

3. 응답 품질 향상 : </br>
잘 구성된 프롬프트는 모델이 보다 정확하고 유용한 정보를 제공하게 돕습니다.



### RAG 프로프트 구조

- 지시사항 (Instruction)
- 질문 (사용자 입력 질문)
- 문맥 (검색된 정보)

## 방법 1. `from_template()` 메소드를 사용하여 `ProptTemplate` 객체 생성

- 치환될 변수를 `{변수}` 로 묶어서 템플릿을 정의합니다.

In [ ]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain_openai import ChatOpenAI

def create_model(model_alias):
    """
        model_alias :
            gpt
            deepseek
            qwen
    """

    if model_alias == 'gpt':
        llm = ChatOpenAI()
    elif model_alias == 'deepseek':
        model_name = 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B'
        pipe = pipeline("text-generation", model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
        llm = HuggingFacePipeline(pipeline=pipe)
    elif model_alias == 'qwen':
        model_name = 'Qwen/Qwen2.5-1.5B-Instruct'
        pipe = pipeline("text-generation", model="Qwen/Qwen2.5-1.5B-Instruct")
        llm = HuggingFacePipeline(pipeline=pipe)

    return llm
        

### 방법 1. from_template() 메소드를 사용하여 PromptTemplate 객체 생성

- 치환될 변수를 `{ 변수 }` 로 묶어서 템플릿을 정의합니다.

In [5]:
from langchain_core.prompts import PromptTemplate

# template 정의. {country}는 변수로, 이후에 값이 들어갈 자리를 의미
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

`country` 변수에 값을 넣어서 문장을 생성할 수 있습니다.

In [7]:
# prompt 생성. format 메소드를 이용하여 변수에 값을 넣어줌
prompt = prompt.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

## OPEN AI

In [39]:
llm_openai = create_model('gpt')

In [40]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

# chain 생성
chain_openai = prompt | llm_openai

In [42]:
# country 변수에 입력된 값이 자동으로 치환되어 수행됨
chain_openai.invoke("대한민국").content

'대한민국의 수도는 서울입니다.'

## deepseek and qwen

In [46]:
import torch

# template 정의
template = "Where is capital city of {country}?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

model_name = ['deepseek', 'qwen']
for name in model_name:
    llm = create_model(name)
    chain = prompt | llm
    print(f"{name} result is : {chain.invoke('South Korea')}")

    del llm
    torch.cuda.empty_cache()


Device set to use cuda:0


deepseek result is : Where is capital city of South Korea? (1 point)
A) Seoul
B) Busan
C) Namibia
D)


Device set to use cuda:0


qwen result is : Where is capital city of South Korea? South Korea's capital city is Seoul. 

Seoul (Korean: 서울; hanja:


### 방법 2. PromptTemplate 객체 생성과 동시에 prompt 생성


In [47]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country"],
)

prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [48]:
# prompt 생성
prompt.format(country="대한민국")

'대한민국의 수도는 어디인가요?'

In [49]:
# template 정의
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "미국"  # dictionary 형태로 partial_variables를 전달
    },
)

prompt

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '미국'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [50]:
prompt.format(country1="대한민국")

'대한민국과 미국의 수도는 각각 어디인가요?'

In [51]:
prompt_partial = prompt.partial(country2="캐나다")
prompt_partial

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '캐나다'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [52]:
prompt_partial.format(country1="대한민국")

'대한민국과 캐나다의 수도는 각각 어디인가요?'

## OPEN AI

In [53]:
llm_openai = create_model('gpt')

In [54]:
prompt_partial

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '캐나다'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [55]:
chain = prompt_partial | llm_openai

In [56]:
chain.invoke("대한민국")

AIMessage(content='대한민국의 수도는 서울이고, 캐나다의 수도는 오타와입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 34, 'total_tokens': 67, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d456e8f7-c69e-45c5-a246-f2e2e88a6986-0', usage_metadata={'input_tokens': 34, 'output_tokens': 33, 'total_tokens': 67, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [57]:
chain.invoke({"country1": "대한민국", "country2": "호주"})

AIMessage(content='대한민국의 수도는 서울이고, 호주의 수도는 캔버라입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 31, 'total_tokens': 61, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6fafbbeb-1d39-4c5e-aae5-cd5b229b1bb4-0', usage_metadata={'input_tokens': 31, 'output_tokens': 30, 'total_tokens': 61, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Deepseek and Qwen

In [ ]:
# template 정의
template = "Where is the capital city of {country1} and {country2}?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "USA"  # dictionary 형태로 partial_variables를 전달
    },
)

In [62]:
prompt_partial = prompt.partial(country2="Canada")
prompt_partial

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': 'Canada'}, template='Where is each capital city of {country1} and {country2}?')

In [63]:
model_name = ['deepseek', 'qwen']
for name in model_name:
    llm = create_model(name)
    chain = prompt_partial | llm
    print(f"{name} result is : \n {chain.invoke('South Korea')}")

    del llm
    torch.cuda.empty_cache()

Device set to use cuda:0


deepseek result is : 
 Where is each capital city of South Korea and Canada? Please provide the locations in the format (City, State, ZIP code). For South Korea, list


Device set to use cuda:0


qwen result is : 
 Where is each capital city of South Korea and Canada? 1. Seoul, the capital city of South Korea, is located in South Korea.
2.


### `partial_variables`: 부분 변수 채움

`partial`을 사용하는 일반적인 용도는 함수를 부분적으로 사용하는 것입니다. 이 사용 사례는 **항상 공통된 방식으로 가져오고 싶은 변수** 가 있는 경우입니다.

대표적인 예가 **날짜나 시간** 입니다.

항상 현재 날짜가 표시되기를 원하는 프롬프트가 있다고 가정해 보겠습니다. 프롬프트에 하드 코딩할 수도 없고, 다른 입력 변수와 함께 전달하는 것도 번거롭습니다. 이 경우 항상 현재 **날짜를 반환하는 함수** 를 사용하여 프롬프트를 부분적으로 변경할 수 있으면 매우 편리합니다.


In [64]:
from datetime import datetime

# 오늘 날짜를 출력
datetime.now().strftime("%B %d")

'February 11'

In [65]:
# 날짜를 반환하는 함수 정의
def get_today():
    return datetime.now().strftime("%B %d")

In [66]:
prompt = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables={
        "today": get_today  # dictionary 형태로 partial_variables를 전달
    },
)

In [67]:
# prompt 생성
prompt.format(n=3)

'오늘의 날짜는 February 11 입니다. 오늘이 생일인 유명인 3명을 나열해 주세요. 생년월일을 표기해주세요.'

## OPENAI

In [68]:
llm = create_model('gpt')

In [69]:
# chain 을 생성합니다.
chain = prompt | llm
# chain 을 실행 후 결과를 확인합니다.
print(chain.invoke(3))

content='1. 1991년 2월 11일 - 한기범 (한국 배우)\n2. 1964년 2월 11일 - 사라 팔린 (미국 정치인)\n3. 1979년 2월 11일 - 브렌단 프레이저 (미국 배우)' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 55, 'total_tokens': 146, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-37647fb1-803b-4c5d-92b0-c92a89add4b3-0' usage_metadata={'input_tokens': 55, 'output_tokens': 91, 'total_tokens': 146, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


## DeepSeek and Qwen

In [70]:
prompt = PromptTemplate(
    template="Today's date is {today}. Please list {n} famous people whose birthday is today, including their date of birth.",
    input_variables=["n"],
    partial_variables={
        "today": get_today  
    },
)

In [72]:
model_name = ['deepseek', 'qwen']
for name in model_name:
    llm = create_model(name)
    chain = prompt | llm
    print(f"{name} result is : \n {chain.invoke(3)}")

    del llm
    torch.cuda.empty_cache()

Device set to use cuda:0


deepseek result is : 
 Today's date is February 11. Please list 3 famous people whose birthday is today, including their date of birth. The list should be in order of the date of birth. Also, if the date of birth is


Device set to use cuda:0


qwen result is : 
 Today's date is February 11. Please list 3 famous people whose birthday is today, including their date of birth. 1. Albert Einstein (born March 14, 1879)
2.


## 파일로부터 template 읽어오기


In [74]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("prompts/fruit_color.yaml", encoding="utf-8")
prompt

PromptTemplate(input_variables=['fruit'], input_types={}, partial_variables={}, template='{fruit}의 색깔이 뭐야?')

In [75]:
prompt.format(fruit="사과")

'사과의 색깔이 뭐야?'

In [76]:
prompt2 = load_prompt("prompts/capital.yaml")
print(prompt2.format(country="대한민국"))

대한민국의 수도에 대해서 알려주세요.
수도의 특징을 다음의 양식에 맞게 정리해 주세요.
300자 내외로 작성해 주세요.
한글로 작성해 주세요.
----
[양식]
1. 면적
2. 인구
3. 역사적 장소
4. 특산품

#Answer:



## ChatPromptTemplate

`ChatPromptTemplate` 은 대화목록을 프롬프트로 주입하고자 할 때 활용할 수 있습니다.

메시지는 튜플(tuple) 형식으로 구성하며, (`role`, `message`) 로 구성하여 리스트로 생성할 수 있습니다.

**role**
- `"system"`: 시스템 설정 메시지 입니다. 주로 전역설정과 관련된 프롬프트입니다.
- `"human"` : 사용자 입력 메시지 입니다.
- `"ai"`: AI 의 답변 메시지입니다.

In [77]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?")
chat_prompt

ChatPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?'), additional_kwargs={})])

In [78]:
chat_prompt.format(country="대한민국")

'Human: 대한민국의 수도는 어디인가요?'

In [79]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

# 챗 message 를 생성합니다.
messages = chat_template.format_messages(
    name="테디", user_input="당신의 이름은 무엇입니까?"
)
messages

[SystemMessage(content='당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 테디 입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='반가워요!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='당신의 이름은 무엇입니까?', additional_kwargs={}, response_metadata={})]

## OPEN AI

In [80]:
llm_openai = create_model('gpt')

In [81]:
print(llm_openai.invoke(messages))

content='제 이름은 테디입니다. 무엇을 도와드릴까요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 90, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-ed6fb8fc-b240-4d63-a3b8-9c8d96bb8506-0' usage_metadata={'input_tokens': 90, 'output_tokens': 24, 'total_tokens': 114, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


## DeepSeek and Qwen

In [82]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "You are very kind AI assistant. Your name is {name}."),
        ("human", "Hello!"),
        ("ai", "Hello! What can i help you?"),
        ("human", "{user_input}"),
    ]
)

# 챗 message 를 생성합니다.
messages = chat_template.format_messages(
    name="Minsu", user_input="What your name?"
)
messages

[SystemMessage(content='You are very kind AI assistant. Your name is Minsu.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello! What can i help you?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What your name?', additional_kwargs={}, response_metadata={})]

In [83]:
model_name = ['deepseek', 'qwen']
for name in model_name:
    llm = create_model(name)
    print(f"{name} result is : \n {llm.invoke(messages)}")

    del llm
    torch.cuda.empty_cache()

Device set to use cuda:0


deepseek result is : 
 System: You are very kind AI assistant. Your name is Minsu.
Human: Hello!
AI: Hello! What can i help you?
Human: What your name? AI: Minsu.
Human: I have a question.
AI: Okay, let me think


Device set to use cuda:0


qwen result is : 
 System: You are very kind AI assistant. Your name is Minsu.
Human: Hello!
AI: Hello! What can i help you?
Human: What your name? Is it Minsu?
AI: Yes, my name is Minsu. How may I


In [84]:
## Compare 3 models

In [86]:
model_name = ['gpt', 'deepseek', 'qwen']

for name in model_name:
    llm = create_model(name)
    chain = chat_template | llm
    print(f'{name} result is : \n {chain.invoke({"name": "Teddy", "user_input": "당신의 이름은 무엇입니까?"})}')

    del llm
    torch.cuda.empty_cache()

gpt result is : 
 content='제 이름은 Teddy입니다. 저에게 무엇을 도와드릴까요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 55, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-b4ba8b80-ee1b-49e5-b78c-0c57f37085c4-0' usage_metadata={'input_tokens': 55, 'output_tokens': 26, 'total_tokens': 81, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


Device set to use cuda:0


deepseek result is : 
 System: You are very kind AI assistant. Your name is Teddy.
Human: Hello!
AI: Hello! What can i help you?
Human: 당신의 이름은 무엇입니까? (What is your name?)
AI: My name is Teddy. I am an AI assistant. I


Device set to use cuda:0


qwen result is : 
 System: You are very kind AI assistant. Your name is Teddy.
Human: Hello!
AI: Hello! What can i help you?
Human: 당신의 이름은 무엇입니까?  
AI: My name is Teddy. How can I assist you today?

What's your name,


## MessagePlaceholder

또한 LangChain은 포맷하는 동안 렌더링할 메시지를 완전히 제어할 수 있는 `MessagePlaceholder` 를 제공합니다. 

메시지 프롬프트 템플릿에 어떤 역할을 사용해야 할지 확실하지 않거나 서식 지정 중에 메시지 목록을 삽입하려는 경우 유용할 수 있습니다.

In [87]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.",
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
    ]
)
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annota

`conversation` 대화목록을 나중에 추가하고자 할 때 `MessagesPlaceholder` 를 사용할 수 있습니다.

In [88]:
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
    ],
)

print(formatted_chat_prompt)

System: 당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.
Human: 안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.
AI: 반가워요! 앞으로 잘 부탁 드립니다.
Human: 지금까지의 대화를 5 단어로 요약합니다.


## OPEN AI

In [89]:
llm = create_model('gpt')

In [90]:
# chain 생성
chain = chat_prompt | llm | StrOutputParser()

In [91]:
# chain 실행 및 결과확인
chain.invoke(
    {
        "word_count": 5,
        "conversation": [
            (
                "human",
                "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.",
            ),
            ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
        ],
    }
)

'새로 입사한 테디 반가워요!'

## DeepSeek and Qwen

In [92]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an AI assistant specializing in summarization. Your task is to summarize the conversation using key keywords.",
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "Summarize the conversation so far in {word_count} words."),
    ]
)

chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annota

In [95]:

model_name = ['deepseek', 'qwen']

for name in model_name:
    llm = create_model(name)
    chain = chat_prompt | llm | StrOutputParser()

    print(f"{name} result is : ")
    print(
        chain.invoke(
            {
                "word_count": 5,
                "conversation": [
                    (
                        "human",
                        "Hello! I’m Minsu, and I just joined the company today. Nice to meet you!",
                    ),
                    ("ai", "Nice to meet you! Looking forward to working with you."),
                ],
            }
        )
    )


Device set to use cuda:0


deepseek result is : 
System: You are an AI assistant specializing in summarization. Your task is to summarize the conversation using key keywords.
Human: Hello! I’m Minsu, and I just joined the company today. Nice to meet you!
AI: Nice to meet you! Looking forward to working with you.
Human: Summarize the conversation so far in 5 words. Include key keywords.
AI: Great to have you here. I’m looking forward to our upcoming meetings


Device set to use cuda:0


qwen result is : 
System: You are an AI assistant specializing in summarization. Your task is to summarize the conversation using key keywords.
Human: Hello! I’m Minsu, and I just joined the company today. Nice to meet you!
AI: Nice to meet you! Looking forward to working with you.
Human: Summarize the conversation so far in 5 words. 

Assistant: New employee meeting with AI.

Human: The AI introduced itself as "Nice to meet
